# Imports

In [1]:
import os
import subprocess
import sys

os.environ["GEOMSTATS_BACKEND"] = "pytorch"

# build work path from git root path
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()  # code/my28brains/
code_dir = os.path.dirname(work_dir)  # code/
raw_dir = "/home/data/28andMeOC_correct"
project_dir = os.path.join(
    os.getcwd(), "project_menstrual"
)  # code/my28brains/project_regression/
src_dir = os.path.join(os.getcwd(), "src")
h2_dir = os.path.join(os.getcwd(), "H2_SurfaceMatch")
sys.path.append(code_dir)
sys.path.append(h2_dir)

In [2]:
import itertools
import os
import random

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pyvista as pv

# import meshplot as mp
from IPython.display import clear_output, display
from ipywidgets import interact, widgets
from mpl_toolkits.mplot3d import Axes3D, art3d

os.environ["GEOMSTATS_BACKEND"] = "pytorch"  # noqa: E402
import geomstats.backend as gs

import src.setcwd

src.setcwd.main()


import H2_SurfaceMatch.utils.input_output as h2_io
import H2_SurfaceMatch.utils.utils
import project_menstrual.default_config as default_config
import src.datasets.utils as data_utils
from H2_SurfaceMatch.utils.input_output import plotGeodesic
from src.regression import check_euclidean, training

/var/folders/yw/6dl955ps7fj9ysnf7qh0j4tc0000gn/T/ipykernel_40941/551281012.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
INFO: Using pytorch backend


Working directory:  /Users/adelemyers/code/my28brains/src
Directory added to path:  /Users/adelemyers/code/my28brains
Directory added to path:  /Users/adelemyers/code/my28brains/src
Directory added to path:  /Users/adelemyers/code/my28brains/H2_SurfaceMatch
Directory added to path:  /Users/adelemyers/code/my28brains/src/notebooks
Directory added to path:  /Users/adelemyers/code/my28brains/src/notebooks/csv
[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


# Multiple Linear Regression

In [3]:
# multiple linear regression, as done in project_menstrual/main_2_linear_regression

(
    space,
    y,
    all_hormone_levels,
    true_intercept,
    true_coef,
) = data_utils.load_real_data(default_config)

n_vertices = len(y[0])
faces = gs.array(space.faces).numpy()

n_train = int(default_config.train_test_split * len(y))

X_indices = np.arange(len(y))
# Shuffle the array to get random values
random.shuffle(X_indices)
train_indices = X_indices[:n_train]
train_indices = np.sort(train_indices)
test_indices = X_indices[n_train:]
test_indices = np.sort(test_indices)

# TODO: instead, save these values in main_2, and then load them here. or, figure out how to predict the mesh using just the intercept and coef learned here, and then load them.

progesterone_levels = gs.array(all_hormone_levels["Prog"].values)
estrogen_levels = gs.array(all_hormone_levels["Estro"].values)
dheas_levels = gs.array(all_hormone_levels["DHEAS"].values)
lh_levels = gs.array(all_hormone_levels["LH"].values)
fsh_levels = gs.array(all_hormone_levels["FSH"].values)
shbg_levels = gs.array(all_hormone_levels["SHBG"].values)

X_multiple = gs.vstack(
    (
        progesterone_levels,
        estrogen_levels,
        dheas_levels,
        lh_levels,
        fsh_levels,
        shbg_levels,
    )
).T  # NOTE: copilot thinks this should be transposed.

(
    multiple_intercept_hat,
    multiple_coef_hat,
    mr,
) = training.fit_linear_regression(y, X_multiple)

mr_score_array = training.compute_R2(y, X_multiple, test_indices, train_indices)

X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))
y_pred_for_mr = mr.predict(X_multiple_predict)
y_pred_for_mr = y_pred_for_mr.reshape([len(X_multiple), n_vertices, 3])

<module 'project_menstrual.default_config' from '/Users/adelemyers/code/my28brains/project_menstrual/default_config.py'>
project_dir: /Users/adelemyers/code/my28brains/project_menstrual
Using menstrual mesh data (from reparameterized directory)

e. (Sort) Found 29 .plys for (left, -1) in /Users/adelemyers/code/my28brains/project_menstrual/results/1_preprocess/d_reparameterized
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
vertices.shape  (1146, 3)
[Open3D WARNING] Read PLY failed: unable to open file: /Users/adelemyers/code/my28brains/project_menstrual/results/1_preproces

RPly: Unable to open file


# Parameters for sliders

In [4]:
hormones_info = {
    "progesterone": {"min_value": 0, "max_value": 15, "step": 1},
    "FSH": {"min_value": 0, "max_value": 15, "step": 1},
    "LH": {"min_value": 0, "max_value": 50, "step": 5},
    "estrogen": {"min_value": 0, "max_value": 250, "step": 20},
    "DHEAS": {"min_value": 0, "max_value": 300, "step": 20},
    "SHBG": {"min_value": 0, "max_value": 70, "step": 5},
}

In [5]:
import numpy as np
import pandas as pd


def generate_hormone_dataframe(hormones_info):
    data = {
        "min_value": [],
        "max_value": [],
        "step": [],
        "num_points": [],
        "values": [],
    }
    for hormone, info in hormones_info.items():
        min_value = info["min_value"]
        max_value = info["max_value"]
        step = info["step"]
        num_points = int((max_value - min_value) / step) + 1
        values = np.linspace(min_value, max_value, num=num_points)

        data["min_value"].append(min_value)
        data["max_value"].append(max_value)
        data["step"].append(step)
        data["num_points"].append(num_points)
        data["values"].append(values)

    df = pd.DataFrame(data, index=hormones_info.keys())
    return df


# Generate the dataframe
hormone_df = generate_hormone_dataframe(hormones_info)

print(hormone_df)

              min_value  max_value  step  num_points  \
progesterone          0         15     1          16   
FSH                   0         15     1          16   
LH                    0         50     5          11   
estrogen              0        250    20          13   
DHEAS                 0        300    20          16   
SHBG                  0         70     5          15   

                                                         values  
progesterone  [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...  
FSH           [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...  
LH            [0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...  
estrogen      [0.0, 20.833333333333332, 41.666666666666664, ...  
DHEAS         [0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0, 14...  
SHBG          [0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...  


# Load meshes for all possible hormone combinations

In [6]:
print(hormones_info.keys())

data = {
    "x": [],
    "y": [],
    "z": [],
    "i": [],
    "j": [],
    "k": [],
}

for progesterone, estrogen, DHEAS, LH, FSH, SHBG in itertools.product(
    hormone_df.loc['progesterone', 'values'], 
    hormone_df.loc['estrogen','values'], 
    hormone_df.loc['DHEAS','values'], 
    hormone_df.loc['LH','values'], 
    hormone_df.loc['FSH','values'], 
    hormone_df.loc['SHBG','values']
):
    
    # Predict Mesh
    X_multiple = gs.vstack(
        (
            gs.array(progesterone),
            gs.array(estrogen),
            gs.array(DHEAS),
            gs.array(LH),
            gs.array(FSH),
            gs.array(SHBG),
        )
    ).T
    X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))
    y_pred_for_mr = mr.predict(X_multiple_predict)
    y_pred_for_mr = y_pred_for_mr.reshape([n_vertices, 3])
    faces = gs.array(space.faces).numpy()

    # Extract x, y, z values
    x = y_pred_for_mr[:, 0]
    y = y_pred_for_mr[:, 1]
    z = y_pred_for_mr[:, 2]

    # Extract i, j, k values
    i = faces[:, 0]
    j = faces[:, 1]
    k = faces[:, 2]

    # Append results to list
    results.append([x, y, z, i, j, k])

# Create DataFrame from results
columns = ["x", "y", "z", "i", "j", "k"]
results_df = pd.DataFrame(results, columns=columns)

# Display the DataFrame
print(results_df)

In [1]:
# THIS WORKS BUT WILL KILL YOUR NOTEBOOK

# results_dir = os.path.join(default_config.project_dir, "results")
# anim_dir = os.path.join(results_dir, "3_animation")
# sys.path.append(anim_dir)

# if not os.path.exists(anim_dir):
#         os.makedirs(anim_dir)

# Assuming you have defined progesterone, estrogen, DHEAS, LH, FSH, SHBG, mr, n_vertices, and space elsewhere

# Initialize dictionary to store pre-calculated values
# pre_calculated_values = {}

# # Generate all possible combinations of hormone values
# combinations = list(itertools.product(
#     hormone_df.loc['progesterone', 'values'], 
#     hormone_df.loc['estrogen','values'], 
#     hormone_df.loc['DHEAS','values'], 
#     hormone_df.loc['LH','values'], 
#     hormone_df.loc['FSH','values'], 
#     hormone_df.loc['SHBG','values']
# ))


# # Loop over each combination of hormone values
# for combination in combinations:
#     print(combination)
#     # Extract hormone values
#     progesterone, estrogen, DHEAS, LH, FSH, SHBG = combination

#     mesh_path = os.path.join(anim_dir, f"progesterone{progesterone}_estrogen{estrogen}_DHEAS{DHEAS}_LH{LH}_FSH{FSH}_SHBG{SHBG}_mesh.csv")

#     if not os.path.exists(mesh_path):
#         # Predict Mesh and Save
#         data = {
#             "x": [],
#             "y": [],
#             "z": [],
#             "i": [],
#             "j": [],
#             "k": [],
#         }
        
#         X_multiple = gs.vstack((
#             gs.array(progesterone),
#             gs.array(estrogen),
#             gs.array(DHEAS),
#             gs.array(LH),
#             gs.array(FSH),
#             gs.array(SHBG),
#         )).T
#         X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))
#         y_pred_for_mr = mr.predict(X_multiple_predict)
#         y_pred_for_mr = y_pred_for_mr.reshape([n_vertices, 3])
#         faces = gs.array(space.faces).numpy()
            
#         # Extract x, y, z values
#         x = y_pred_for_mr[:, 0]
#         y = y_pred_for_mr[:, 1]
#         z = y_pred_for_mr[:, 2]
            
#         # Extract i, j, k values
#         i = faces[:, 0]
#         j = faces[:, 1]
#         k = faces[:, 2]
    
#         print(f"len x {len(x)}")
#         print(f"len i {len(i)}")
        
#         # Store pre-calculated values in dictionary
#         data["x"].append(x)
#         data["y"].append(y)
#         data["z"].append(z)
#         data["i"].append(i)
#         data["j"].append(j)
#         data["k"].append(k)
    
#         columns = ["x", "y", "z", "i", "j", "k"]
#         mesh_df = pd.DataFrame(data)
#         mesh_df.to_csv(mesh_path)
    
#         # check to make sure it can be opened
#         df = pd.read_csv(mesh_path)
#         display(df)
#         x_values = df["x"]

# Plot hormones via slider

In [ ]:
def plot_hormone_levels_plotly(progesterone, FSH, LH, estrogen, SHBG, DHEAS):
    progesterone = gs.array(progesterone)
    FSH = gs.array(FSH)
    LH = gs.array(LH)
    estrogen = gs.array(estrogen)
    SHBG = gs.array(SHBG)
    DHEAS = gs.array(DHEAS)

    # Predict Mesh
    X_multiple = gs.vstack(
        (
            progesterone,
            estrogen,
            DHEAS,
            LH,
            FSH,
            SHBG,
        )
    ).T

    X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))

    y_pred_for_mr = mr.predict(X_multiple_predict)
    print(y_pred_for_mr.shape)
    y_pred_for_mr = y_pred_for_mr.reshape([n_vertices, 3])

    faces = gs.array(space.faces).numpy()

    x = y_pred_for_mr[:, 0]
    y = y_pred_for_mr[:, 1]
    z = y_pred_for_mr[:, 2]

    i = faces[:, 0]
    j = faces[:, 1]
    k = faces[:, 2]

    fig = go.Figure(
        data=[
            go.Mesh3d(
                x=x,
                y=y,
                z=z,
                colorbar_title="z",
                colorscale=[[0, "gold"], [0.5, "mediumturquoise"], [1, "magenta"]],
                # Intensity of each vertex, which will be interpolated and color-coded
                # intensity=[0, 0.33, 0.66, 1],
                # i, j and k give the vertices of triangles
                # here we represent the 4 triangles of the tetrahedron surface
                i=i,
                j=j,
                k=k,
                name="y",
                showscale=True,
            )
        ]
    )

    fig.update_layout(width=1000)
    fig.update_layout(height=800)

    fig.show()

# Create Sliders and Plot

In [ ]:
# Function to create sliders
def create_slider(min_value, max_value, step, description):
    return widgets.FloatSlider(
        value=0.5, min=min_value, max=max_value, step=step, description=description
    )


# Create sliders using data from hormone_df
progesterone_slider = create_slider(
    hormone_df.loc["progesterone", "min_value"],
    hormone_df.loc["progesterone", "max_value"],
    hormone_df.loc["progesterone", "step"],
    "Progesterone",
)

FSH_slider = create_slider(
    hormone_df.loc["FSH", "min_value"],
    hormone_df.loc["FSH", "max_value"],
    hormone_df.loc["FSH", "step"],
    "FSH",
)

LH_slider = create_slider(
    hormone_df.loc["LH", "min_value"],
    hormone_df.loc["LH", "max_value"],
    hormone_df.loc["LH", "step"],
    "LH",
)

estrogen_slider = create_slider(
    hormone_df.loc["estrogen", "min_value"],
    hormone_df.loc["estrogen", "max_value"],
    hormone_df.loc["estrogen", "step"],
    "Estrogen",
)

DHEAS_slider = create_slider(
    hormone_df.loc["DHEAS", "min_value"],
    hormone_df.loc["DHEAS", "max_value"],
    hormone_df.loc["DHEAS", "step"],
    "DHEAS",
)

SHBG_slider = create_slider(
    hormone_df.loc["SHBG", "min_value"],
    hormone_df.loc["SHBG", "max_value"],
    hormone_df.loc["SHBG", "step"],
    "SHBG",
)


interact(
    plot_hormone_levels_plotly,
    progesterone=progesterone_slider,
    FSH=FSH_slider,
    LH=LH_slider,
    estrogen=estrogen_slider,
    SHBG=SHBG_slider,
    DHEAS=DHEAS_slider,
    # fits=fixed(fits),
)

In [ ]:
# def plot_hormone_levels(change):
#     with output:
#         clear_output(wait=True)
#         # Retrieve the current values of the sliders
#         progesterone = gs.array(progesterone_slider.value)
#         FSH = gs.array(FSH_slider.value)
#         LH = gs.array(LH_slider.value)
#         estrogen = gs.array(estrogen_slider.value)
#         SHBG = gs.array(SHBG_slider.value)
#         DHEAS = gs.array(DHEAS_slider.value)

#         # Predict Mesh
#         X_multiple = gs.vstack(
#             (
#                 progesterone,
#                 estrogen,
#                 DHEAS,
#                 LH,
#                 FSH,
#                 SHBG,
#             )
#         ).T

#         X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))

#         y_pred_for_mr = mr.predict(X_multiple_predict)
#         print(y_pred_for_mr.shape)
#         y_pred_for_mr = y_pred_for_mr.reshape([len(y[0]), 3])


#         # Decimate mesh for plotting
#         vertices = y_pred_for_mr
#         faces = gs.array(space.faces).numpy()
#         # n_faces_after_decimation = int(faces.shape[0] / 110)
#         # vertices, faces = H2_SurfaceMatch.utils.utils.decimate_mesh(
#         #     vertices, faces, n_faces_after_decimation
#         # )

#         print(vertices.shape)

#         # Plotting
#         mesh = pv.PolyData(vertices, faces)
#         mesh.plot(show_edges=True, line_width=5)
#         # mp.plot(vertices, faces)

In [ ]:
# def plot_hormone_levels(progesterone, FSH, LH, estrogen, SHBG, DHEAS, faces):
#     progesterone = gs.array(progesterone)
#     FSH = gs.array(FSH)
#     LH = gs.array(LH)
#     estrogen = gs.array(estrogen)
#     SHBG = gs.array(SHBG)
#     DHEAS = gs.array(DHEAS)

#     # Predict Mesh
#     X_multiple = gs.vstack(
#         (
#             progesterone,
#             estrogen,
#             DHEAS,
#             LH,
#             FSH,
#             SHBG,
#         )
#     ).T

#     X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))

#     y_pred_for_mr = mr.predict(X_multiple_predict)
#     print(y_pred_for_mr.shape)
#     y_pred_for_mr = y_pred_for_mr.reshape([len(y[0]), 3])

#     # Decimate mesh for plotting
#     vertices = y_pred_for_mr
#     # n_faces_after_decimation = int(faces.shape[0] / 110)
#     # vertices, faces = H2_SurfaceMatch.utils.utils.decimate_mesh(
#     #     vertices, faces, n_faces_after_decimation
#     # )

#     print(vertices.shape)

#     # Plotting
#     mesh = pv.PolyData(vertices, faces)
#     mesh.plot(show_edges=True, line_width=5)

In [ ]:
# def plot_hormone_levels_matplotlib(
#     progesterone, FSH, LH, estrogen, SHBG, DHEAS, n_vertices, faces
# ):
#     progesterone = gs.array(progesterone)
#     FSH = gs.array(FSH)
#     LH = gs.array(LH)
#     estrogen = gs.array(estrogen)
#     SHBG = gs.array(SHBG)
#     DHEAS = gs.array(DHEAS)

#     # Predict Mesh
#     X_multiple = gs.vstack(
#         (
#             progesterone,
#             estrogen,
#             DHEAS,
#             LH,
#             FSH,
#             SHBG,
#         )
#     ).T

#     X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))

#     y_pred_for_mr = mr.predict(X_multiple_predict)
#     print(y_pred_for_mr.shape)
#     y_pred_for_mr = y_pred_for_mr.reshape([n_vertices, 3])

#     v = y_pred_for_mr
#     f = gs.array(space.faces).numpy()
#     C = np.array([1])

#     # x = y_pred_for_mr[:, 0]
#     # y = y_pred_for_mr[:, 1]
#     # z = y_pred_for_mr[:, 2]

#     # print(x)
#     # print(y)
#     # print(z)

#     fig = plt.figure()
#     ax = fig.add_subplot(projection="3d")

#     norm = plt.Normalize(C.min(), C.max())
#     colors = plt.cm.viridis(norm(C))

#     pc = art3d.Poly3DCollection(v[f], facecolors=colors, edgecolor="black")
#     ax.add_collection(pc)

#     ax.set_xlim(-3, 3)
#     ax.set_ylim(-3, 3)
#     ax.set_zlim(-3, 3)

#     plt.show()

# # Plot the surface
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')

# ax.plot_trisurf(x, y, z)

# # Set an equal aspect ratio
# ax.set_aspect('equal')

# plt.show()

# fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50)])
# fig.show()

In [ ]:
# def plot_hormone_levels_static_plot(progesterone, FSH, LH, estrogen, SHBG, DHEAS):
#     progesterone = gs.array(progesterone)
#     FSH = gs.array(FSH)
#     LH = gs.array(LH)
#     estrogen = gs.array(estrogen)
#     SHBG = gs.array(SHBG)
#     DHEAS = gs.array(DHEAS)

#     # Predict Mesh
#     X_multiple = gs.vstack(
#         (
#             progesterone,
#             estrogen,
#             DHEAS,
#             LH,
#             FSH,
#             SHBG,
#         )
#     ).T

#     X_multiple_predict = gs.array(X_multiple.reshape(len(X_multiple), -1))

#     y_pred_for_mr = mr.predict(X_multiple_predict)
#     print(y_pred_for_mr.shape)
#     y_pred_for_mr = y_pred_for_mr.reshape([len(y[0]), 3])

#     # Decimate mesh for plotting
#     vertices = y_pred_for_mr
#     faces = gs.array(space.faces).numpy()
#     # n_faces_after_decimation = int(faces.shape[0] / 110)
#     # vertices, faces = H2_SurfaceMatch.utils.utils.decimate_mesh(
#     #     vertices, faces, n_faces_after_decimation
#     # )

#     vertices = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0], [0.5, 0.5, -1]])

#     # mesh faces
#     faces = np.hstack(
#         [
#             [4, 0, 1, 2, 3],  # square
#             [3, 0, 1, 4],  # triangle
#             [3, 1, 2, 4],  # triangle
#         ]
#     )

#     surf = pv.PolyData(vertices, faces)

#     # plot each face with a different color
#     surf.plot(
#         scalars=np.arange(3),
#         cpos=[-1, 1, 0.5],
#         show_scalar_bar=False,
#         show_edges=True,
#         line_width=5,
#     )

In [ ]:
# Create widgets for the four hormones
# progesterone_slider = widgets.FloatSlider(value=0.5, min=0, max=10, step=0.01, description='Progesterone')
# FSH_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='FSH')
# LH_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='LH')
# estrogen_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Estrogen')
# DHEAS_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='DHEAS')
# SHBG_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='SHBG')

# output = widgets.Output()

# # Attach the plotting function to the sliders
# progesterone_slider.observe(plot_hormone_levels, names='value')
# FSH_slider.observe(plot_hormone_levels, names='value')
# LH_slider.observe(plot_hormone_levels, names='value')
# estrogen_slider.observe(plot_hormone_levels, names='value')
# DHEAS_slider.observe(plot_hormone_levels, names='value')
# SHBG_slider.observe(plot_hormone_levels, names='value')

# # Display the widgets
# widgets_display = widgets.VBox([progesterone_slider, FSH_slider, LH_slider, estrogen_slider, DHEAS_slider, SHBG_slider, output])
# display(widgets_display)

In [ ]:
# mesh points
vertices = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0], [0.5, 0.5, -1]])

# mesh faces
faces = np.hstack(
    [
        [4, 0, 1, 2, 3],  # square
        [3, 0, 1, 4],  # triangle
        [3, 1, 2, 4],  # triangle
    ]
)

surf = pv.PolyData(vertices, faces)

# plot each face with a different color
surf.plot(
    scalars=np.arange(3),
    cpos=[-1, 1, 0.5],
    show_scalar_bar=False,
    show_edges=True,
    line_width=5,
)